## CREAZIONE DELL'AMBIENTE DI LAVORO

#### ACQUISIZIONE DEL DATASET

In [10]:
import os

# NB: come scritto nel readme questa parte non è fondamentale se ci sono gia' i file
# necessari nella cartella /data/

# Cambiare le credenziali kaggle per poter scaricare i dati
# Si uncommenti e si cambino le stringhe con le proprie credenziali
# os.environ['KAGGLE_USERNAME'] = "your_username"
# os.environ['KAGGLE_KEY'] = "your_password"

# Download di tutti i file .csv presenti nel dataset di instgram
!kaggle datasets download -p ./data -d bhanupratapbiswas/instgram --unzip 

Dataset URL: https://www.kaggle.com/datasets/bhanupratapbiswas/instgram
License(s): ODC Public Domain Dedication and Licence (PDDL)
100%|█████████████████████████████████████████| 151k/151k [00:00<00:00, 523kB/s]
100%|█████████████████████████████████████████| 151k/151k [00:00<00:00, 521kB/s]


#### CONTAINER NEO4J

Per fare il build e lo start del container di neo4j (si parte dalla empty sandbox) si deve far girare nella cartella del progetto contente `docker-compose.yml` il comando riportato di seguito.
```bash
sudo docker-compose up
```

**NB**: Nell'ambiente di visual studio non si riesce, purtroppo, a far girare comandi di questo tipo in modo interattivo (nessuna possibilità di immettere la password).
Si e' anche valutato se mettere la password in un file .env e richiamarla da riga di comando ma non sarebbe stata sicura. Per questo si richiede all'utente di immeterlo da terminale.

Prima di continuare aspettare la conferma dell'avvio del container, come in figura:
![container neo4j](media/container_start.png)

#### CONNESSIONE AL DATABASE

In [17]:
from neo4j import GraphDatabase as gd

# Prima di arrivare a questo punto si controlli di aver correttamente attivato il container
# Nel caso in cui ci siano dubbi su ip/porta da usare si controlli nel log docker alla voce "bolt"
URI = "bolt://localhost:7688" 

d   = gd.driver(URI, auth=None)
d.verify_connectivity()


#### CREAZIONE DATABASE

In [28]:
s   = d.session()

# Prova a creare un nuovo database vuoto col nome di instagram
# Che ci fosse gia' o meno alla fine setta la sessione sul database instagram
# Lo si fa all'inizio per non dover specificare ogni volta su che database fare le query
try:
    s.run("create database instagram").data() 
except: 
    s = d.session(database="instagram")
finally:
    s = d.session(database="instagram")

In [29]:
# La creazione dei constraint viene fatta per ogni nodo che sara' presente
# I constraint in questione specificano tutti l'unicita' dell'ID per ogni nodo
s.run("create constraint UniqueUserID if not exists for (u:User) require u.id is unique;")
s.run("create constraint UniqueCommentID if not exists for (c:Comment) require c.id is unique;")
s.run("create constraint UniquePostID if not exists for (p:Post) require p.id is unique;")
s.run("create constraint UniqueTagID if not exists for (t:Tag) require t.id is unique;")

# Nel caso in cui il constraint esista gia' non verrà creato nessun errore grazie a 'if not exists'


Accedendo a http://localhost:7474 e poi con nessuna autenticazione a bolt://localhost:7688 si può:
* controllare con `:dbs` che esiste il database instagram
* con `:use instagram` entrare nel database e con `:schema` controllare che siano stati aggiunti i quattro constraint

In [30]:
# Si è fatto un test per essere sicuri di come e dove trovare tutti i csv
# La stampa ritorna il numero di righe del file (8782)
res = s.run('''load csv with headers
                from 'file:///likes.csv' as line
                return count(line)''').data()
print(res)

[{'count(line)': 8782}]


#### AGGIUNTA NODI

In [35]:
# Si e' notato che le date di creazione di tag, commenti e foto sono tutte fabbricate
# lo stesso giorno
# Si e' deciso quindi di non usarle in quanto di poca utilita'

# Per i nodi user non vengono messe le properties 'post count' 
# e 'verified status' in quanto la prima si puo' ricavare e la seconda 
# non sembra importante per le analisi che si faranno
s.run('''load csv with headers
         from 'file:///users.csv' as row
         with row merge (u:User{id:toInteger(row.id)})
         on create set
            u.name = row.name,
            u.created = toString(row.`created time`), 
            u.type = toString(row.`private/public`)''')

s.run('''load csv with headers from 'file:///tags.csv' as row 
         with row merge (t:Tag{id:toInteger(row.id)}) 
         on create set 
            t.text = toString(row.`tag text`), 
            t.location = toString(row.location) ''')

# Per le foto non vengono caricati i link in quanto non servono per l'analisi che si fara'
s.run('''load csv with headers from 'file:///photos.csv' as row 
         with row merge (p:Post{id:toInteger(row.id)}) 
         on create set
            p.filter = toString(row.`Insta filter used`),
            p.type = toString(row.`photo type`) ''')


s.run('''load csv with headers from 'file:///comments.csv' as row 
         with row merge (c:Comment{id:toInteger(row.id)}) 
         on create set
            c.text = toString(row.comment) ''')

#### AGGIUNTA RELAZIONI

In [36]:
# Relazione di creazione post da parte dello user
s.run('''load csv with headers from 'file:///photos.csv' as row 
         match (p:Post{id:toInteger(row.id)})
         match (u:User{id:toInteger(row.`user ID`)}) 
         merge (p)<-[:POSTED]-(u) ''') 

# Relazione di un tag contenuto in un post
# La relazione tra tag e user può essere ricavata da user -> post -> tag
s.run('''load csv with headers from 'file:///photo_tags.csv' as row 
         match (p:Post{id:toInteger(row.photo)})
         match (t:Tag{id:toInteger(row.`tag ID`)}) 
         merge (p)-[:HAS_TAG]->(t) ''') 

# Relazione di un like di uno user su un post
# Anche in questo caso il 'created time' e' stato fabbricato e non viene quindi preso in considerazione
# La properties 'following or not' si può invece ricavare e non viene quindi settata
s.run('''load csv with headers from 'file:///likes.csv' as row 
         match (p:Post{id:toInteger(row.photo)})
         match (u:User{id:toInteger(row.user)}) 
         merge (p)<-[r:LIKE]-(u) 
         set r.type = row.`like type `''') 

# Relazione di follow tra user
# Anche in questo caso le properties 'is follower active' e 'followee Acc status' non sono fondamentali
# e possono essere ricavate dalle varie relazioni e properties gia' salvate
s.run('''load csv with headers from 'file:///follows.csv' as row 
         match (u1:User{id:toInteger(row.follower)})
         match (u2:User{id:toInteger(row.followee)}) 
         merge (u1)-[:FOLLOW]->(u2) ''') 

# Relazione di un commento sotto un post
s.run('''load csv with headers from 'file:///comments.csv' as row 
         match (c:Comment{id:toInteger(row.id)})
         match (p:Post{id:toInteger(row.`Photo id`)}) 
         merge (p)<-[r:COMMENT_ON]-(c) ''') 

# Relazione di uno user che crea un commento
s.run('''load csv with headers from 'file:///comments.csv' as row 
         match (c:Comment{id:toInteger(row.id)})
         match (u:User{id:toInteger(row.`User id`)}) 
         merge (c)-[:COMMENTED_BY]->(u) ''') 


#### AGGIUNTA INDICI

In [38]:
# Si sceglie di creare dei text index per un po' tutte le proprieta' dei nodi
# questo perchè tutti i campi sono in formato di stringa e non si pensa di aver mai 
# bisogno di alzare le performance di operazioni numeriche (range indexes)
s.run('''create text index for (u:User) on (u.name)''')
s.run('''create text index for (u:User) on (u.type)''')

s.run('''create text index for (t:Tag) on (t.text)''')
s.run('''create text index for (t:Tag) on (t.location)''')

s.run('''create text index for (p:Post) on (p.type)''')

s.run('''create text index for (c:Comment) on (c.text)''')

# Gli id sono fatti invece di default dai constraint di unicita'

## EXPLORATORY DATA ANALYSIS

## RESEARCH QUESTIONS

In [ ]:
# Ricordiamo sempre di chiudere la sessione aperta
s.close()
d.close()